In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

import warnings
warnings.filterwarnings("ignore")

# LOAD DATASET

In [ ]:
import pandas as pd

df = pd.read_csv("./list-ecommerce-for-delivery-and-review-prediction.csv")

In [ ]:
display(df)

# DUPLIKASI DATA

In [ ]:
exact_duplicate_count = df.duplicated(keep='first').sum()
print("Jumlah Data yang duplikas:", exact_duplicate_count)

exact_duplicate_rows = df[df.duplicated(keep='first')]
display(exact_duplicate_rows)

In [ ]:
df_cleaned = df.drop_duplicates(keep='first')
print("Number of rows after removing duplicates:", len(df_cleaned))

# CEK MISSING VALUES

In [ ]:
print("Null values per column:")
print(df_cleaned.isnull().sum())

# **1. ANALISASI EDA**

### **A. analisis pengaruh ketepatan waktu dengan rating**

In [ ]:
bins = [-float('inf'), -24, 0, 24, 168, float('inf')]
labels = ['Early', 'On-Time', 'Slight Delay', 'Moderate Delay', 'Significant Delay']
df_cleaned.loc[:, 'delivery_delay_range'] = pd.cut(df_cleaned['delivery_delay_hours'], bins=bins, labels=labels, right=False)

print("Value counts for 'delivery_delay_range':")
print(df_cleaned['delivery_delay_range'].value_counts())

In [ ]:
average_review_scores_by_delay = df_cleaned.groupby('delivery_delay_range', observed=True)['review_score'].mean()
display(average_review_scores_by_delay)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
# Use a good colormap for the bars, e.g., 'viridis'
colors = sns.color_palette("viridis", n_colors=len(average_review_scores_by_delay))
sns.barplot(
    x=average_review_scores_by_delay.index,
    y=average_review_scores_by_delay.values,
    palette=colors
)
plt.title('Average Review Score by Delivery Delay Range')
plt.xlabel('Delivery Delay Range')
plt.ylabel('Average Review Score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### **Penyebaran data numerik dan kategorical**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize numerical data distribution
numerical_cols = df_cleaned.select_dtypes(include=['float64', 'int64']).columns
df_cleaned[numerical_cols].hist(bins=15, figsize=(15, 10))
plt.tight_layout()
plt.show()

In [ ]:
# Visualize categorical data distribution in a 2x2 grid
categorical_cols = df_cleaned.select_dtypes(include=['object', 'category']).columns
filtered_categorical_cols = [col for col in categorical_cols if df_cleaned[col].nunique() < 20]

n_plots = min(4, len(filtered_categorical_cols))
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, col in enumerate(filtered_categorical_cols[:4]):
    sns.countplot(data=df_cleaned, y=col, order=df_cleaned[col].value_counts().index, ax=axes[i])
    axes[i].set_title(f'Distribution of {col}')
    axes[i].set_xlabel('Count')
    axes[i].set_ylabel(col)

# Hide any unused subplots
for j in range(n_plots, 4):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

### **C.Distribus data target**

In [ ]:
# Create a 1-row, 2-column subplot for review score distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Countplot for review score distribution
sns.countplot(
    data=df_cleaned,
    x='review_score',
    order=df_cleaned['review_score'].value_counts().index,
    palette='viridis',
    ax=axes[0]
)
axes[0].set_title('Distribution of Review Scores (Countplot)')
axes[0].set_xlabel('Review Score')
axes[0].set_ylabel('Count')

# Boxplot for review score distribution
sns.boxplot(
    data=df_cleaned,
    y='review_score',
    palette='viridis',
    ax=axes[1]
)
axes[1].set_title('Distribution of Review Scores (Boxplot)')
axes[1].set_ylabel('Review Score')
axes[1].set_xlabel('')

plt.tight_layout()
plt.show()

### Distribusi Kolom Numerik

In [ ]:
numeric_cols = [
    'delivery_delay_hours', 'price', 'freight_value',
    'product_description_lenght', 'product_photos_qty',
    'time_to_ship_hours', 'purchase_count', 'avg_review_score'
]

# Statistik deskriptif
descriptive_stats = df[numeric_cols].describe().T
descriptive_stats["mode"] = df[numeric_cols].mode().iloc[0]
print(descriptive_stats)


In [ ]:
plt.figure(figsize=(16, 20))

for i, col in enumerate(numeric_cols):
    plt.subplot(4, 2, i + 1)
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f'Distribusi: {col}')

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(16, 20))
plt.suptitle('Boxplots of Numeric Columns', fontsize=18, y=1.02)
for i, col in enumerate(numeric_cols):
    plt.subplot(4, 2, i + 1)
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot: {col}')
    
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

### **Visualisasi keterkaitan harga barang dan biaya ongkir dengan rating**

**Reasoning**:
Create scatter plots to visualize the relationship between price and review score, and freight value and review score.



In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df_cleaned, x='price', y='review_score', alpha=0.5)
plt.title('Review Score vs Price')
plt.xlabel('Price')
plt.ylabel('Review Score')
plt.show()

plt.figure(figsize=(12, 6))
sns.scatterplot(data=df_cleaned, x='freight_value', y='review_score', alpha=0.5)
plt.title('Review Score vs Freight Value')
plt.xlabel('Freight Value')
plt.ylabel('Review Score')
plt.show()

In [ ]:
price_bins = [0, 50, 100, 200, 500, 1000, float('inf')]
price_labels = ['0-50', '51-100', '101-200', '201-500', '501-1000', '>1000']
df_cleaned['price_range'] = pd.cut(df_cleaned['price'], bins=price_bins, labels=price_labels, right=False)

freight_bins = [0, 10, 20, 30, 50, float('inf')]
freight_labels = ['0-10', '11-20', '21-30', '31-50', '>50']
df_cleaned['freight_range'] = pd.cut(df_cleaned['freight_value'], bins=freight_bins, labels=freight_labels, right=False)

average_review_scores_by_price = df_cleaned.groupby('price_range')['review_score'].mean()
display(average_review_scores_by_price)

average_review_scores_by_freight = df_cleaned.groupby('freight_range')['review_score'].mean()
display(average_review_scores_by_freight)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(
    x=average_review_scores_by_price.index,
    y=average_review_scores_by_price.values,
    palette=sns.color_palette("YlGnBu", n_colors=len(average_review_scores_by_price))
)
plt.title('Average Review Score by Price Range')
plt.xlabel('Price Range')
plt.ylabel('Average Review Score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(
    x=average_review_scores_by_freight.index,
    y=average_review_scores_by_freight.values,
    palette=sns.color_palette("YlOrRd", n_colors=len(average_review_scores_by_freight))
)
plt.title('Average Review Score by Freight Value Range')
plt.xlabel('Freight Value Range')
plt.ylabel('Average Review Score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

### **Hubungan Seller grade, sekker state dan product category dengan rating**

In [ ]:
average_review_scores_by_seller_grade = df_cleaned.groupby('seller_grade')['review_score'].mean().sort_values()
# display(average_review_scores_by_seller_grade)

In [ ]:
df_cleaned['same_state'] = (df_cleaned['seller_state'] == df_cleaned['customer_state'])

average_delivery_delay_by_state = df_cleaned.groupby('same_state')['delivery_delay_hours'].mean()
average_review_score_by_state = df_cleaned.groupby('same_state')['review_score'].mean()

print("Average Delivery Delay Hours by State Match:")
display(average_delivery_delay_by_state)

In [ ]:
print("\nAverage Review Score by State Match:")
display(average_review_score_by_state)

In [ ]:
average_review_scores_by_category = df_cleaned.groupby('product_category_name_english')['review_score'].mean().sort_values()
display(average_review_scores_by_category)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(
    x=average_review_scores_by_seller_grade.index,
    y=average_review_scores_by_seller_grade.values,
    palette="viridis"
)
plt.title('Average Review Score by Seller Grade')
plt.xlabel('Seller Grade')
plt.ylabel('Average Review Score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
sns.barplot(
    x=average_review_score_by_state.index,
    y=average_review_score_by_state.values,
    palette="mako"
)
plt.title('Average Review Score by Seller/Customer Same State')
plt.xlabel('Seller and Customer in Same State')
plt.ylabel('Average Review Score')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Use a visually appealing color palette for the barplots
fig, axes = plt.subplots(2, 1, figsize=(10, 16))

# Top 5 (descending order)
top5 = average_review_scores_by_category.sort_values(ascending=False).head(5)
sns.barplot(
    y=top5.index,
    x=top5.values,
    ax=axes[0],
    palette="crest"
)
axes[0].set_title('Top 5 Product Categories by Average Review Score', fontsize=16, fontweight='bold')
axes[0].set_xlabel('Average Review Score', fontsize=13)
axes[0].set_ylabel('Product Category', fontsize=13)
axes[0].tick_params(axis='y', labelsize=12)
axes[0].tick_params(axis='x', labelsize=12)

# Bottom 5 (ascending order)
bottom5 = average_review_scores_by_category.sort_values(ascending=True).head(5)
sns.barplot(
    y=bottom5.index,
    x=bottom5.values,
    ax=axes[1],
    palette="flare"
)
axes[1].set_title('Bottom 5 Product Categories by Average Review Score', fontsize=16, fontweight='bold')
axes[1].set_xlabel('Average Review Score', fontsize=13)
axes[1].set_ylabel('Product Category', fontsize=13)
axes[1].tick_params(axis='y', labelsize=12)
axes[1].tick_params(axis='x', labelsize=12)

plt.tight_layout()
plt.show()

### **Korelasi setiap feature dengan rating Score**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Calculate correlation matrix for numeric columns
correlation_matrix = df_cleaned.corr(numeric_only=True)

# --- 1. Correlation with Review Score (Barplot) ---
corr_to_review = correlation_matrix['review_score'].drop('review_score').sort_values(key=abs, ascending=False)

plt.figure(figsize=(8, 6))
sns.barplot(
    y=corr_to_review.index,
    x=corr_to_review.values,
    palette="vlag"
)
plt.title('Feature Correlation with Review Score', fontsize=16, fontweight='bold')
plt.xlabel('Correlation Coefficient', fontsize=13)
plt.ylabel('Feature', fontsize=13)
plt.axvline(0, color='gray', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

In [ ]:

mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

plt.figure(figsize=(12, 10))
sns.heatmap(
    correlation_matrix,
    mask=mask,
    annot=True,
    fmt=".2f",
    cmap='Spectral',
    linewidths=0.7,
    cbar_kws={"shrink": 0.8, "label": "Correlation"},
    square=True,
    annot_kws={"size": 10}
)
plt.title('Correlation Matrix of Numeric Features', fontsize=18, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right', fontsize=11)
plt.yticks(rotation=0, fontsize=11)
plt.tight_layout()
plt.show()

# Preprocessing

In [ ]:
df_model = df_cleaned.copy()

# Target: 0 = Bad (<3), 1 = Neutral (3-4), 2 = Good (>4)
def map_review_score(score):
    if score < 3:
        return 0  # Bad
    elif 3 <= score < 4:
        return 1  # Neutral
    else:
        return 2  # Good

df_model['sentiment_score'] = df_model['review_score'].apply(map_review_score)
print(df_model['sentiment_score'].value_counts())

In [ ]:
df_model.columns

In [ ]:
# Prepare features and target for regression
# Target: avg_review_score (regression)

# Drop columns that should not be used as features
drop_cols = [
    'review_score',         # original review score (if present)
    'avg_review_score',     # target
    'sentiment_score',      # classification target
    'order_id', 'product_id', 'customer_id', 'seller_id', "customer_unique_id",# IDs
    'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date',
    'order_delivered_customer_date', 'order_estimated_delivery_date'
]
# Only drop columns that exist
drop_cols = [col for col in drop_cols if col in df_model.columns]

X = df_model.drop(columns=drop_cols)
y = df_model['avg_review_score']

In [ ]:

label_encoders = {}

# Identify categorical columns
categorical_features = ['seller_state', 'customer_state', 'seller_grade', 'product_category_name_english', 
                        'delivery_delay_range', 'price_range', 'freight_range', "same_state"]

# Apply LabelEncoder to each categorical column
for feature in categorical_features:
    le = LabelEncoder()
    X[feature] = le.fit_transform(X[feature])
    label_encoders[feature] = le  # Store the encoder for future use

# Display the transformed dataframe
display(X.head())

In [ ]:
X.columns

# MACHINE LEARNING

In [ ]:
!pip install tabm rtdl-num-embeddings

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## TabM (Tabular Deep Learning with Parameter-Efficient Ensembling)

TabM is a state-of-the-art tabular deep learning model that efficiently imitates an ensemble of MLPs through:

1. **Parameter-Efficient Ensembling**: Uses weight sharing between k submodels for efficiency
2. **Parallel Training**: All k submodels are trained simultaneously  
3. **Feature Embeddings**: Uses PiecewiseLinearEmbeddings for better numerical feature representation
4. **Proper Loss Function**: Each of the k predictions is trained independently (mean loss, not loss of mean)

Key advantages:
- Better performance than traditional MLPs on tabular data
- More efficient than full ensembles due to weight sharing
- Handles both numerical and categorical features effectively

In [ ]:
# Scale features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Helper function for evaluation
def eval_regression(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2 Score: {r2:.4f}")
    return {'mae': mae, 'rmse': rmse, 'r2': r2}

# Dictionary to hold results
results = {}

# 1. Linear Regression
print('--- Linear Regression ---')
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
results['LinearRegression'] = eval_regression(lr, X_test_scaled, y_test)

# 2. Random Forest Regressor
print('\n--- Random Forest Regressor ---')
rf = RandomForestRegressor(n_estimators=500, random_state=42)
rf.fit(X_train_scaled, y_train)
results['RandomForest'] = eval_regression(rf, X_test_scaled, y_test)

# 3. XGBoost Regressor
print('\n--- XGBoost Regressor ---')
if device == 'cuda':
    xgb = XGBRegressor(n_estimators=500, tree_method='gpu_hist', predictor='gpu_predictor', random_state=42)
else:
    xgb = XGBRegressor(n_estimators=500, random_state=42)
xgb.fit(X_train_scaled, y_train)
results['XGBoost'] = eval_regression(xgb, X_test_scaled, y_test)

# 4. Decision Tree Regressor
print('\n--- Decision Tree Regressor ---')
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train_scaled, y_train)
results['DecisionTree'] = eval_regression(dt, X_test_scaled, y_test)


In [ ]:
import matplotlib.pyplot as plt

# Extract model names and metrics
model_names = list(results.keys())
mae_values = [results[model]['mae'] for model in model_names]
rmse_values = [results[model]['rmse'] for model in model_names]
r2_values = [results[model]['r2'] for model in model_names]

# Create a bar plot for MAE, RMSE, and R2
x = range(len(model_names))
width = 0.25

plt.figure(figsize=(12, 6))

# Plot MAE
plt.bar(x, mae_values, width=width, label='MAE', color='skyblue')

# Plot RMSE
plt.bar([i + width for i in x], rmse_values, width=width, label='RMSE', color='orange')

# Plot R2
plt.bar([i + 2 * width for i in x], r2_values, width=width, label='R2', color='green')

# Add labels and title
plt.xlabel('Models')
plt.ylabel('Metrics')
plt.title('Comparison of Model Performance')
plt.xticks([i + width for i in x], model_names, rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Save the best model based on R2 score
best_model_name = max(results, key=lambda model: results[model]['r2'])
print(f"Best model based on R2 score: {best_model_name}")

# Get the best model object
if best_model_name == 'LinearRegression':
    best_model = lr
elif best_model_name == 'RandomForest':
    best_model = rf
elif best_model_name == 'XGBoost':
    best_model = xgb
elif best_model_name == 'DecisionTree':
    best_model = dt
elif best_model_name == 'TabM':
    best_model = tabm_model
else:
    raise ValueError(f"Unknown model name: {best_model_name}")

import joblib
if best_model_name == 'TabM':
    # For PyTorch models, save the state dict
    torch.save({
        'model_state_dict': best_model.state_dict(),
        'model_config': {
            'n_num_features': X_train_scaled.shape[1],
            'd_out': 1,
            'k': 32,
            'n_blocks': 3,
            'd_block': 256,
            'dropout': 0.1
        }
    }, f"best_model_{best_model_name}.pth")
    print(f"Best model saved as best_model_{best_model_name}.pth")
else:
    joblib.dump(best_model, f"best_model_{best_model_name}.pkl")
    print(f"Best model saved as best_model_{best_model_name}.pkl")

In [ ]:
# 5. TabM Regressor (Optimized for Tabular Data)
print('\n--- TabM Regressor ---')
from tabm import TabM
from rtdl_num_embeddings import PiecewiseLinearEmbeddings

# Create TabM with feature embeddings for better performance
tabm_model = TabM.make(
    n_num_features=X_train_scaled.shape[1],
    d_out=1,  # Regression task
    num_embeddings=PiecewiseLinearEmbeddings(
        n_features=X_train_scaled.shape[1],
        n_bins=64,  # Number of bins for piecewise linear embeddings
        d_embedding=16  # Embedding dimension
    ),
    k=32,  # Ensemble size
    n_blocks=3,  # Number of blocks in the MLP backbone
    d_block=256,  # Hidden dimension
    dropout=0.1,  # Dropout rate
)

if device == 'cuda':
    tabm_model = tabm_model.cuda()

# Training setup for TabM
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_train_tensor = torch.FloatTensor(y_train.values)
y_test_tensor = torch.FloatTensor(y_test.values)

if device == 'cuda':
    X_train_tensor = X_train_tensor.cuda()
    X_test_tensor = X_test_tensor.cuda()
    y_train_tensor = y_train_tensor.cuda()
    y_test_tensor = y_test_tensor.cuda()

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

# Optimizer
optimizer = optim.AdamW(tabm_model.parameters(), lr=0.002, weight_decay=0.0003)

# Loss function for regression
def tabm_loss_fn(y_pred, y_true):
    # TabM produces k predictions. Each must be trained separately.
    # y_pred shape: (batch_size, k, 1) for regression
    # y_true shape: (batch_size,)
    
    # Flatten predictions: (batch_size, k, 1) -> (batch_size * k,)
    y_pred_flat = y_pred.squeeze(-1).flatten()
    
    # Repeat targets for each ensemble member: (batch_size,) -> (batch_size * k,)
    y_true_repeated = y_true.repeat_interleave(tabm_model.backbone.k)
    
    return nn.functional.mse_loss(y_pred_flat, y_true_repeated)

# Training loop
tabm_model.train()
n_epochs = 100
best_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(n_epochs):
    epoch_loss = 0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        y_pred = tabm_model(batch_x)
        loss = tabm_loss_fn(y_pred, batch_y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    
    # Early stopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.6f}')

# Evaluation
tabm_model.eval()
with torch.no_grad():
    y_pred_tabm = tabm_model(X_test_tensor)
    # Average the k predictions for final prediction
    y_pred_final = y_pred_tabm.mean(dim=1).squeeze().cpu().numpy()
    y_test_np = y_test_tensor.cpu().numpy()

# Calculate metrics
mae_tabm = mean_absolute_error(y_test_np, y_pred_final)
rmse_tabm = mean_squared_error(y_test_np, y_pred_final, squared=False)
r2_tabm = r2_score(y_test_np, y_pred_final)

print(f"MAE: {mae_tabm:.4f}")
print(f"RMSE: {rmse_tabm:.4f}")
print(f"R2 Score: {r2_tabm:.4f}")

results['TabM'] = {'mae': mae_tabm, 'rmse': rmse_tabm, 'r2': r2_tabm}

In [ ]:
# Save LabelEncoders
for feature, encoder in label_encoders.items():
    joblib.dump(encoder, f"label_encoder_{feature}.pkl")
    print(f"LabelEncoder for {feature} saved as label_encoder_{feature}.pkl")

# Save MinMaxScaler
joblib.dump(scaler, "minmax_scaler.pkl")
print("MinMaxScaler saved as minmax_scaler.pkl")